In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import xgboost as xgb

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# print(os.listdir("../input"))
data_path = '../input/'

# Any results you write to the current directory are saved as output.

df_train = pd.read_csv(data_path + 'train.csv')
df_test = pd.read_csv(data_path + 'test.csv')

df_train = df_train.sample(n=5000000)

print(df_train.shape)
print(df_test.shape)

(5000000, 8)
(9914, 7)


In [2]:
df_train.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
24320907,2009-05-16 15:11:00.00000066,3.70,2009-05-16 15:11:00 UTC,-73.985632,40.762920,-73.981820,40.768595,1
23865743,2009-12-11 21:50:23.0000002,3.70,2009-12-11 21:50:23 UTC,-73.891110,40.754933,-73.877980,40.756280,1
54931,2012-12-13 14:40:14.0000001,19.00,2012-12-13 14:40:14 UTC,0.000000,0.000000,0.000000,0.000000,1
32499346,2011-08-15 14:15:20.0000004,9.30,2011-08-15 14:15:20 UTC,-73.980400,40.763200,-73.991400,40.750700,2
9491516,2013-04-22 11:03:00.000000195,37.83,2013-04-22 11:03:00 UTC,-73.865890,40.770742,-73.997782,40.729495,1


In [3]:
df_train.dtypes

key                   object
fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

In [4]:
df_train.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,5.000000e+06,5.000000e+06,5.000000e+06,4.999969e+06,4.999969e+06,5.000000e+06
mean,1.133670e+01,-7.251660e+01,3.992458e+01,-7.251906e+01,3.992619e+01,1.684909e+00
std,9.826900e+00,1.271353e+01,8.929744e+00,1.232942e+01,9.705292e+00,1.330812e+00
min,-1.050000e+02,-3.356778e+03,-3.413983e+03,-3.394483e+03,-3.114278e+03,0.000000e+00
25%,6.000000e+00,-7.399207e+01,4.073494e+01,-7.399141e+01,4.073404e+01,1.000000e+00
50%,8.500000e+00,-7.398181e+01,4.075266e+01,-7.398016e+01,4.075315e+01,1.000000e+00
75%,1.250000e+01,-7.396708e+01,4.076713e+01,-7.396370e+01,4.076810e+01,2.000000e+00
max,1.273310e+03,3.442185e+03,3.347260e+03,3.442185e+03,3.400392e+03,2.080000e+02


In [5]:
df_train.isnull().sum()

key                   0
fare_amount           0
pickup_datetime       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude    31
dropoff_latitude     31
passenger_count       0
dtype: int64

In [6]:
df_train = df_train.dropna()
df_train.isnull().sum()

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [7]:
df_test.dtypes

key                   object
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

In [8]:
df_test.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000
mean,-73.974722,40.751041,-73.973657,40.751743,1.671273
std,0.042774,0.033541,0.039072,0.035435,1.278747
min,-74.252193,40.573143,-74.263242,40.568973,1.000000
25%,-73.992501,40.736125,-73.991247,40.735254,1.000000
50%,-73.982326,40.753051,-73.980015,40.754065,1.000000
75%,-73.968013,40.767113,-73.964059,40.768757,2.000000
max,-72.986532,41.709555,-72.990963,41.696683,6.000000


In [9]:
df_test.isnull().sum()

key                  0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [10]:
print(df_train.shape)
print(df_test.shape)

(4999969, 8)
(9914, 7)


In [11]:
df_train = df_train.loc[(df_train['fare_amount'] > 0) & (df_train['fare_amount'] < 200)]
df_train = df_train.loc[(df_train['pickup_longitude'] > -300) & (df_train['pickup_longitude'] < 300)]
df_train = df_train.loc[(df_train['pickup_latitude'] > -300) & (df_train['pickup_latitude'] < 300)]
df_train = df_train.loc[(df_train['dropoff_longitude'] > -300) & (df_train['dropoff_longitude'] < 300)]
df_train = df_train.loc[(df_train['dropoff_latitude'] > -300) & (df_train['dropoff_latitude'] < 300)]
df_train = df_train.loc[df_train['passenger_count'] <= 8]

df_train.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
24320907,2009-05-16 15:11:00.00000066,3.70,2009-05-16 15:11:00 UTC,-73.985632,40.762920,-73.981820,40.768595,1
23865743,2009-12-11 21:50:23.0000002,3.70,2009-12-11 21:50:23 UTC,-73.891110,40.754933,-73.877980,40.756280,1
54931,2012-12-13 14:40:14.0000001,19.00,2012-12-13 14:40:14 UTC,0.000000,0.000000,0.000000,0.000000,1
32499346,2011-08-15 14:15:20.0000004,9.30,2011-08-15 14:15:20 UTC,-73.980400,40.763200,-73.991400,40.750700,2
9491516,2013-04-22 11:03:00.000000195,37.83,2013-04-22 11:03:00 UTC,-73.865890,40.770742,-73.997782,40.729495,1


In [12]:
df_test = df_test.loc[(df_test['pickup_longitude'] > -300) & (df_test['pickup_longitude'] < 300)]
df_test = df_test.loc[(df_test['pickup_latitude'] > -300) & (df_test['pickup_latitude'] < 300)]
df_test = df_test.loc[(df_test['dropoff_longitude'] > -300) & (df_test['dropoff_longitude'] < 300)]
df_test = df_test.loc[(df_test['dropoff_latitude'] > -300) & (df_test['dropoff_latitude'] < 300)]
df_test = df_test.loc[df_test['passenger_count'] <= 8]

df_test.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1


In [13]:
df_train = df_train.reset_index()
df_train.head()

,index,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24320907,2009-05-16 15:11:00.00000066,3.70,2009-05-16 15:11:00 UTC,-73.985632,40.762920,-73.981820,40.768595,1
1,23865743,2009-12-11 21:50:23.0000002,3.70,2009-12-11 21:50:23 UTC,-73.891110,40.754933,-73.877980,40.756280,1
2,54931,2012-12-13 14:40:14.0000001,19.00,2012-12-13 14:40:14 UTC,0.000000,0.000000,0.000000,0.000000,1
3,32499346,2011-08-15 14:15:20.0000004,9.30,2011-08-15 14:15:20 UTC,-73.980400,40.763200,-73.991400,40.750700,2
4,9491516,2013-04-22 11:03:00.000000195,37.83,2013-04-22 11:03:00 UTC,-73.865890,40.770742,-73.997782,40.729495,1


In [14]:
df_train = df_train.drop(['index'], axis=1)
df_train.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-05-16 15:11:00.00000066,3.70,2009-05-16 15:11:00 UTC,-73.985632,40.762920,-73.981820,40.768595,1
1,2009-12-11 21:50:23.0000002,3.70,2009-12-11 21:50:23 UTC,-73.891110,40.754933,-73.877980,40.756280,1
2,2012-12-13 14:40:14.0000001,19.00,2012-12-13 14:40:14 UTC,0.000000,0.000000,0.000000,0.000000,1
3,2011-08-15 14:15:20.0000004,9.30,2011-08-15 14:15:20 UTC,-73.980400,40.763200,-73.991400,40.750700,2
4,2013-04-22 11:03:00.000000195,37.83,2013-04-22 11:03:00 UTC,-73.865890,40.770742,-73.997782,40.729495,1


In [15]:
ids = df_test['key']
train_Y = df_train['fare_amount']

df_train = df_train.drop(['fare_amount'], axis=1)

print(df_train.shape)
print(df_test.shape)

(4999212, 7)
(9914, 7)


In [16]:
df_train.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-05-16 15:11:00.00000066,2009-05-16 15:11:00 UTC,-73.985632,40.762920,-73.981820,40.768595,1
1,2009-12-11 21:50:23.0000002,2009-12-11 21:50:23 UTC,-73.891110,40.754933,-73.877980,40.756280,1
2,2012-12-13 14:40:14.0000001,2012-12-13 14:40:14 UTC,0.000000,0.000000,0.000000,0.000000,1
3,2011-08-15 14:15:20.0000004,2011-08-15 14:15:20 UTC,-73.980400,40.763200,-73.991400,40.750700,2
4,2013-04-22 11:03:00.000000195,2013-04-22 11:03:00 UTC,-73.865890,40.770742,-73.997782,40.729495,1


In [17]:
df_train['pickup_datetime'] = df_train['pickup_datetime'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S UTC'))
df_train['pickup_year'] = df_train['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%Y')).astype('int64')
df_train['pickup_month'] = df_train['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%m')).astype('int64')
df_train['pickup_day'] = df_train['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%d')).astype('int64')
df_train['pickup_hour'] = df_train['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%H')).astype('int64')
df_train['pickup_minute'] = df_train['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%M')).astype('int64')
df_train['pickup_second'] = df_train['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%S')).astype('int64')

df_test['pickup_datetime'] = df_test['pickup_datetime'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S UTC'))
df_test['pickup_year'] = df_test['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%Y')).astype('int64')
df_test['pickup_month'] = df_test['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%m')).astype('int64')
df_test['pickup_day'] = df_test['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%d')).astype('int64')
df_test['pickup_hour'] = df_test['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%H')).astype('int64')
df_test['pickup_minute'] = df_test['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%M')).astype('int64')
df_test['pickup_second'] = df_test['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%S')).astype('int64')

In [18]:
df_train = df_train.drop(['pickup_datetime'], axis=1)
df_test = df_test.drop(['pickup_datetime'], axis=1)

In [19]:
df_train['longitude_diff'] = df_train['dropoff_longitude'] - df_train['pickup_longitude']
df_train['latitude_diff'] = df_train['dropoff_latitude'] - df_train['pickup_latitude']
df_train['distance'] = ((df_train['longitude_diff']**2) + (df_train['latitude_diff']**2))**0.5

df_test['longitude_diff'] = df_test['dropoff_longitude'] - df_test['pickup_longitude']
df_test['latitude_diff'] = df_test['dropoff_latitude'] - df_test['pickup_latitude']
df_test['distance'] = ((df_test['longitude_diff']**2) + (df_test['latitude_diff']**2))**0.5

In [20]:
df_train = df_train.drop(['key'], axis=1)
df_test = df_test.drop(['key'], axis=1)

In [21]:
print(df_train.shape)
print(df_test.shape)

(4999212, 14)
(9914, 14)


In [22]:
df_train.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,pickup_month,pickup_day,pickup_hour,pickup_minute,pickup_second,longitude_diff,latitude_diff,distance
0,-73.985632,40.762920,-73.981820,40.768595,1,2009,5,16,15,11,0,0.003812,0.005675,0.006836
1,-73.891110,40.754933,-73.877980,40.756280,1,2009,12,11,21,50,23,0.013130,0.001347,0.013199
2,0.000000,0.000000,0.000000,0.000000,1,2012,12,13,14,40,14,0.000000,0.000000,0.000000
3,-73.980400,40.763200,-73.991400,40.750700,2,2011,8,15,14,15,20,-0.011000,-0.012500,0.016651
4,-73.865890,40.770742,-73.997782,40.729495,1,2013,4,22,11,3,0,-0.131892,-0.041247,0.138191


In [23]:
df_test.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,pickup_month,pickup_day,pickup_hour,pickup_minute,pickup_second,longitude_diff,latitude_diff,distance
0,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,27,13,8,24,-0.008110,-0.019970,0.021554
1,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,27,13,8,24,-0.012024,0.019817,0.023180
2,-73.982524,40.751260,-73.979654,40.746139,1,2011,10,8,11,53,44,0.002870,-0.005121,0.005870
3,-73.981160,40.767807,-73.990448,40.751635,1,2012,12,1,21,12,12,-0.009288,-0.016172,0.018649
4,-73.966046,40.789775,-73.988565,40.744427,1,2012,12,1,21,12,12,-0.022519,-0.045348,0.050631


In [24]:
# scaler = MinMaxScaler()

# train_X = scaler.fit_transform(df_train)
# test_X = scaler.fit_transform(df_test)

In [25]:
x_train,x_test,y_train,y_test = train_test_split(df_train, train_Y, test_size=0.2, random_state=0)

def xgbmodel(x_train,x_test,y_train,y_test):
    matrix_train = xgb.DMatrix(x_train, label=y_train)
    matrix_test = xgb.DMatrix(x_test, label=y_test)
    model = xgb.train(params={'objective':'reg:linear', 'eval_metric':'rmse'},
                     dtrain=matrix_train,
                     num_boost_round=200,
                     early_stopping_rounds=20,
                     evals=[(matrix_test,'test')])
    return model

myxgbmodel = xgbmodel(x_train,x_test,y_train,y_test)

pred = myxgbmodel.predict(xgb.DMatrix(df_test), ntree_limit=myxgbmodel.best_ntree_limit)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	test-rmse:10.6679
Will train until test-rmse hasn't improved in 20 rounds.
[1]	test-rmse:8.12448
[2]	test-rmse:6.50522
[3]	test-rmse:5.5089
[4]	test-rmse:4.92897
[5]	test-rmse:4.60809
[6]	test-rmse:4.42134
[7]	test-rmse:4.31827
[8]	test-rmse:4.24768
[9]	test-rmse:4.2099
[10]	test-rmse:4.16728
[11]	test-rmse:4.15364
[12]	test-rmse:4.13564
[13]	test-rmse:4.12509
[14]	test-rmse:4.10231
[15]	test-rmse:4.08816
[16]	test-rmse:4.0809
[17]	test-rmse:4.07878
[18]	test-rmse:4.0678
[19]	test-rmse:4.06088
[20]	test-rmse:4.04884
[21]	test-rmse:4.04368
[22]	test-rmse:4.04174
[23]	test-rmse:4.03563
[24]	test-rmse:4.02771
[25]	test-rmse:4.02551
[26]	test-rmse:4.0234
[27]	test-rmse:4.02242
[28]	test-rmse:4.01882
[29]	test-rmse:4.00943
[30]	test-rmse:4.00423
[31]	test-rmse:3.99867
[32]	test-rmse:3.99835
[33]	test-rmse:3.99702
[34]	test-rmse:3.99313
[35]	test-rmse:3.99072
[36]	test-rmse:3.99026
[37]	test-rmse:3.99021
[38]	test-rmse:3.98944
[39]	test-rmse:3.98829
[40]	test-rmse:3.98558
[41]	test-rmse:

In [26]:
# model = LinearRegression()
# model.fit(df_train, train_Y)
# pred = model.predict(df_test)

In [27]:
pred

array([ 9.8809185, 10.783232 ,  5.1298623, ..., 53.50969  , 19.888685 ,
        7.554152 ], dtype=float32)

In [28]:
submission = pd.DataFrame({'key':ids, 'fare_amount':pred})
submission.to_csv('submission.csv', index=False)